## First, enable BigQuery Connection API. Also, if you are running everything from Notebooks, please make sure the account listed by 'gcloud auth list' has roles "BigQuery Admin" and "Cloud Functions Admin" in order to successfully execute all steps

! bq mk --connection --display_name='remote_cloud_function_connection' --connection_type=CLOUD_RESOURCE --project_id=<your-project-id> --location=<your-location-like-US> remote_cloudfunc_conn

## You can retrieve the service account identifier, which you can do by using the bq command-line tool as shown in the following example (some output columns have been omitted for clarity of the example).

In [ ]:
! bq show --location=US --connection  remote_cloudfunc_conn

## Create a BQML penguin model following steps from https://cloud.google.com/bigquery-ml/docs/linear-regression-tutorial

## Export the model using BQ Console (https://cloud.google.com/bigquery-ml/docs/exporting-models#exporting_models) 

## Import the exported model into Vertex AI (https://cloud.google.com/vertex-ai/docs/model-registry/import-model#pre-built-container). Select "Import model artifacts into a new pre-built container" and the pick TensorFlow as the model framework and latest version. Select the gcs location where the exported model is located (example: gs://ca_export_bqmodels/penguin-mass/).

## Deploy an online endpoint on the newly imported model (https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-console#custom-trained)

## Open main.py and update these lines to reflect your setup (endpoint id is found at Vertex AI endpoint details):
    project="<change-me>"
    location="<your-region>"
    endpoint="<your-vertexai-online-endpoint>"

## Using IAM, assign role "AI Platform Developer" to the following service accounts:
- The one listed as output of command (bq show --location=US --connection ...). One of the steps above
- The Cloud Function service account (found under the Cloud Function details tab)

## Deploy the Cloud Function

In [ ]:
! gcloud functions deploy ai_handler \
--runtime python310 --trigger-http

### The "gcloud auth list" user running notebook needs to have the permission bigquery.routines.create on the dataset where you create the remote function, and bigquery.connections.delegate permission (e.g. via BigQuery Connection Admin role) on the connection which is used by the remote function. BigQuery Admin role should do it as well.

## Create the BQ remote function, pointing to the newly deploy Cloud Function. Please update the project, dataset, connection id and endpoint parameters accordinly

In [ ]:
%%bigquery 
CREATE OR REPLACE FUNCTION `<project>.<dataset>.predict_penguin_weight` (species STRING, island STRING, sex STRING, culmen_length_mm FLOAT64, culmen_depth_mm FLOAT64, flipper_length_mm FLOAT64) RETURNS STRING 
REMOTE WITH CONNECTION `<your-connection-name>` 
OPTIONS 
(endpoint = 'https://<your-location>.cloudfunctions.net/ai_handler', 
 user_defined_context = [("mode","penguin_weight")]
)
;

### Test your remote function
Hint: If the function does not return in less than 10 seconds, check the Cloud Function logs and look for errors

In [ ]:
%%bigquery
select <dataset>.predict_penguin_weight("Adelie Penguin (Pygoscelis adeliae)","Dream","MALE",56.5,21,204)


### You can now run some more advanced SQL statements, joining the function

In [ ]:
%%bigquery

with penguins as(
select * from `bigquery-public-data.ml_datasets.penguins` LIMIT 1
)
select body_mass_g as actual_weight, 
<dataset>.predict_penguin_weight(species,
                                  island,
                                  sex,
                                  culmen_length_mm,
                                  culmen_depth_mm,
                                  flipper_length_mm) as predicted_weight
FROM
  penguins;